####Loading the Data from Kaggle

In [1]:
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tobiasbueck/multilingual-customer-support-tickets")

print("Path to dataset files:", path)

100%|██████████| 16.1M/16.1M [00:01<00:00, 12.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/tobiasbueck/multilingual-customer-support-tickets/versions/14


In [3]:
import os

path_to_dataset = "/root/.cache/kagglehub/datasets/tobiasbueck/multilingual-customer-support-tickets/versions/14"

os.listdir(path_to_dataset)

['dataset-tickets-german_normalized.csv',
 'dataset-tickets-german_normalized_50_5_2.csv',
 'aa_dataset-tickets-multi-lang-5-2-50-version.csv',
 'dataset-tickets-multi-lang-4-20k.csv',
 'dataset-tickets-multi-lang3-4k.csv']

In [4]:
BASE_DIR = "/root/.cache/kagglehub/datasets/tobiasbueck/multilingual-customer-support-tickets/versions/14/dataset-tickets-multi-lang-4-20k.csv"

df = pd.read_csv(BASE_DIR)

print(df.head(20))

                                              subject  \
0   Unvorhergesehener Absturz der Datenanalyse-Pla...   
1                            Customer Support Inquiry   
2                       Data Analytics for Investment   
3                  Krankenhaus-Dienstleistung-Problem   
4                                            Security   
5   Concerns About Securing Medical Data on 2-in-1...   
6   Ratung für Sicherung medizinischer Daten in Hu...   
7                            Problem with Integration   
8                                  Assistance Request   
9                                     Support Request   
10  Issue with Data Analytics Platform - Insuffici...   
11  Reported Issue with Project Sync Resulting in ...   
12                     Strategies for Brand Expansion   
13             Issue with Website Analytics Dashboard   
14  Urgent: Data Breach Identified in Hospital Net...   
15       Request for Assistance with Data Integration   
16  Request for Updating Integr

####Creating the data format for the tokenization

In [5]:
df["subject"] = df["subject"].fillna("")
df["body"] = df["body"].fillna("")

df["Final_Text"] = df["subject"] + " " + df["body"]

In [6]:
df = df.dropna(subset=["queue"])

df = df[df["Final_Text"].str.strip() != ""]

In [9]:
print("Subject: \n", df["subject"], "\n")
print("Body \n", df["body"], "\n")
print("Final Text \n", df["Final_Text"], "\n")

Subject: 
 0        Unvorhergesehener Absturz der Datenanalyse-Pla...
1                                 Customer Support Inquiry
2                            Data Analytics for Investment
3                       Krankenhaus-Dienstleistung-Problem
4                                                 Security
                               ...                        
19995       Assistance Needed for IFTTT Docker Integration
19996          Bitten um Unterstützung bei der Integration
19997                                                     
19998              Hilfe bei digitalen Strategie-Problemen
19999    Optimierung Ihrer Datenanalyse-Plattform erlei...
Name: subject, Length: 20000, dtype: object 

Body 
 0        Die Datenanalyse-Plattform brach unerwartet ab...
1        Seeking information on digital strategies that...
2        I am contacting you to request information on ...
3        Ein Medien-Daten-Sperrverhalten trat aufgrund ...
4        Dear Customer Support, I am reaching out t